In [1]:
import sys
sys.path.insert(0, "..")

import IPython
from IPython.display import display, Image, clear_output
from ipywidgets import interact

import matplotlib.pyplot as plt
import imageio
import numpy as np
import cv2 
import invertedai as iai
from IPython.utils import io
import time

import csv
import pickle
import random
import time

# API key:
iai.add_apikey("WyiIbfxsbPak3HpP2JVNma7jfQQIrflxa4AUS6mC")

In [5]:
NUM_SIMULATIONS = 20
SIMULATION_LENGTH = 200

location_list = [
    #"iai:drake_street_and_pacific_blvd",
    #"iai:dunbar_diversion",
    #"iai:marine_drive_viewpoint_3",
    #"iai:ubc_roundabout",
    "iai:terminal_and_quebec"
]

# Simulate with `initialize`, `drive` and `light` until there are collisions.
for i in range(NUM_SIMULATIONS):
    log_dict = {"agent_attributes": [], "agent_states": [], "traffic_light_states": []}
    
    # pick a location (4 way, signalized intgersection)
    location = random.choice(location_list)
    location_info_response = iai.location_info(location=location)
    rendered_static_map = location_info_response.birdview_image.decode()

    is_traffic_lights = True
    traffic_light_state_history = []
    try:
        light_response = iai.light(location=location)
        traffic_light_states = light_response.traffic_lights_states
        traffic_light_state_history = [traffic_light_states]
        log_dict["traffic_light_states"].append(traffic_light_states)
    except:
        light_response = []
        traffic_light_states = None
        traffic_light_state_history = None
        is_traffic_lights = False
    
    num_agents = random.randint(15,25)
    response = iai.initialize(
        location=location,  
        agent_count=num_agents,    
        get_birdview=False,  
        traffic_light_state_history=traffic_light_state_history
    )
    agent_attributes = response.agent_attributes
    log_dict["agent_attributes"] = agent_attributes
    log_dict["agent_states"].append(response.agent_states)

    agent_state_history = []
    traffic_light_state_history = []
    
    is_collision = False
    for _ in range(SIMULATION_LENGTH):  
        print(f"Simulation Number {i}")
        print(f"Number of Agents {num_agents}")
        print(location)
        print(f"Processing step at {_/10}s")
        if is_traffic_lights: 
            light_response = iai.light(location=location, recurrent_states=light_response.recurrent_states)
            traffic_light_states = light_response.traffic_lights_states
            log_dict["traffic_light_states"].append(traffic_light_states)
        else:
            traffic_light_states = None
        response = iai.drive(
            location=location,
            agent_attributes=agent_attributes,
            agent_states=response.agent_states,
            recurrent_states=response.recurrent_states,
            get_birdview=False,
            traffic_lights_states=traffic_light_states,
            get_infractions=True,
            random_seed=random.randint(1,10)
        )
        for inf in response.infractions:
            is_collision = is_collision or inf.collisions
        log_dict["agent_states"].append(response.agent_states)
        clear_output(wait=True)
    
    location_substring = str(location.split(":")[1])
    directory_file_name = ""
    if is_collision:
        directory_file_name = 'blame_example_logs/'+location_substring+'/collision/blame_demo_logs_data_'+str(int(time.time()))+'.pkl'
    else:
        directory_file_name = 'blame_example_logs/'+location_substring+'/nocollision/blame_demo_logs_data_'+str(int(time.time()))+'.pkl'
    with open(directory_file_name, 'wb') as f:
        pickle.dump(log_dict, f)
            
clear_output(wait=True)
print("Done")

Simulation Number 12
Number of Agents 21
iai:terminal_and_quebec
Processing step at 3.2s


KeyboardInterrupt: 